## Convolutional Neural Network (CNN)

## Load the libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Load and reshape the data and normalize

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data() # many data set are included in keras

x_train = x_train.reshape(-1,28,28,1)
x_test  = x_test.reshape(-1,28,28,1)

print("x_train : ",x_train.shape)
print("y_train : ",y_train.shape)
print("x_test  : ",x_test.shape)
print("y_test  : ",y_test.shape)

xmax=x_train.max()
x_train = x_train / xmax
x_test  = x_test  / xmax

x_train :  (60000, 28, 28, 1)
y_train :  (60000,)
x_test  :  (10000, 28, 28, 1)
y_test  :  (10000,)


## Creation of CNN model

In [3]:
model_cnn = keras.models.Sequential()

model_cnn.add( keras.layers.Input((28,28,1)) )

model_cnn.add( keras.layers.Conv2D(8, (3,3),  activation='relu') )
model_cnn.add( keras.layers.MaxPooling2D((2,2)))
model_cnn.add( keras.layers.Dropout(0.2))

model_cnn.add( keras.layers.Conv2D(16, (3,3), activation='relu') )
model_cnn.add( keras.layers.MaxPooling2D((2,2)))
model_cnn.add( keras.layers.Dropout(0.2))

model_cnn.add( keras.layers.Flatten()) 
model_cnn.add( keras.layers.Dense(100, activation='relu'))
model_cnn.add( keras.layers.Dropout(0.5))

model_cnn.add( keras.layers.Dense(10, activation='softmax'))

In [4]:
model_cnn.summary()

model_cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 8)         80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 8)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 8)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 16)        1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 5, 5, 16)          0

With the CNN, the number of parameters is divided by 2, almost 90,000 for the DNN against 42 358 for the CNN.

model_cnn.add( keras.layers.Conv2D(8, (3,3),  activation='relu') )
means  a convolution layer with 2 dimensions, the kernel size is (3,3) and the activation fonction is 'relu'. The number of parameters of this layer 8*(3x3+1) '8' for the number of convolution, '3x3' is the dimension of kernel and '1' the bias

## Train the model

In [ ]:
batch_size  = 512
epochs      =  16

history = model_cnn.fit(  x_train, y_train,
                      batch_size      = batch_size,
                      epochs          = epochs,
                      verbose         = 1,
                      validation_data = (x_test, y_test))

Epoch 1/16
118/118 [==============================] - 10s 80ms/step - loss: 1.0776 - accuracy: 0.6438 - val_loss: 0.2679 - val_accuracy: 0.9301
Epoch 2/16
118/118 [==============================] - 9s 72ms/step - loss: 0.3765 - accuracy: 0.8865 - val_loss: 0.1427 - val_accuracy: 0.9583
Epoch 3/16
118/118 [==============================] - 8s 71ms/step - loss: 0.2585 - accuracy: 0.9210 - val_loss: 0.1061 - val_accuracy: 0.9675
Epoch 4/16
118/118 [==============================] - 9s 73ms/step - loss: 0.2139 - accuracy: 0.9353 - val_loss: 0.0870 - val_accuracy: 0.9731
Epoch 5/16
118/118 [==============================] - 8s 71ms/step - loss: 0.1853 - accuracy: 0.9444 - val_loss: 0.0726 - val_accuracy: 0.9771
Epoch 6/16
118/118 [==============================] - 8s 68ms/step - loss: 0.1652 - accuracy: 0.9494 - val_loss: 0.0642 - val_accuracy: 0.9801
Epoch 7/16
118/118 [==============================] - 8s 71ms/step - loss: 0.1526 - accuracy: 0.9538 - val_loss: 0.0586 - val_accuracy: 0.981

## Evaluate the model

In [ ]:
score_cnn = model_cnn.evaluate(x_test, y_test, verbose=0)

print(f'Test loss     : {score_cnn[0]:4.4f}')
print(f'Test accuracy : {score_cnn[1]:4.4f}')

## Plot

With a DNN, we had a precision of the order of : 97.7%

In [ ]:
# convert the training history to a dataframe
history_df = pd.DataFrame(history.history)

# use Pandas native plot method
plt.xlabel("epochs")
plt.ylabel("rate_validity")
plt.title("Rate_validation acoording the number of epochs")
history_df['accuracy'].plot()
history_df['val_accuracy'].plot()

## Plot results

In [ ]:
y_sigmoid = model_cnn.predict(x_test)
y_pred    = np.argmax(y_sigmoid, axis=-1)

cf_matrix=tf.math.confusion_matrix(y_test, y_pred)
print(cf_matrix)
sns.heatmap(cf_matrix, annot=True)

## Plot some errors

In [ ]:
images = xtest.reshape((-1, 28, 28))

# poorly predicted data are recovered 
misclass = (y_test != y_pred)
misclass_images = images[misclass,:,:]
misclass_predicted = y_pred[misclass]

# a sample of these images is selected
select = np.random.randint(misclass_images.shape[0], size=12)

# the images and the (erroneous) predictions associated with these images are displayed
for index, value in enumerate(select):
    plt.subplot(3,4,index+1)
    plt.axis('off')
    plt.imshow(misclass_images[value],cmap=plt.cm.gray_r,interpolation="nearest")
    plt.title('Predicted: {}'.format(misclass_predicted[value]) )

plt.show()